# Sample Exam: Coffee Shops

Java June is a company that owns coffee shops in a number of locations in Europe.

The company knows that stores with more reviews typically get more new customers. This is
because new customers consider the number of reviews when picking between two shops.

They want to get more insight into what leads to more reviews.

They are also interested in whether there is a link between the number of reviews and rating.

They want a report to answer these questions.

# Task 1

Before you start your analysis, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "coffee.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| Region    | Nominal. </br> Where the store is located. One of 10 possible regions (A to J).</br> Missing values should be replaced with “Unknown”.|
| Place name | Nominal. </br>The name of the store. </br>Missing values should be replaced with “Unknown”.|
| Place type  | Nominal. </br>The type of coffee shop. One of “Coffee shop”, “Cafe”, “Espresso bar”, and “Others”. </br>Missing values should be replaced with “Unknown”. |
| Rating   | Ordinal. </br>Average rating of the store from reviews. On a 5 point scale. </br>Missing values should be replaced with 0. |
| Reviews  | Nominal. </br>The number of reviews given to the store. </br>Missing values should be replaced with the overall median number.|
| Price  | Ordinal. </br>The price range of products in the store. One of '\$', '\$\$' or '\$\$\$'. </br>Missing values should be replaced with ”Unknown”.|
| Delivery Option   | Nominal. </br>If delivery is available. Either True or False. </br>Missing values should be replaced with False. |
| Dine in Option | Nominal. </br>If dine in is available. Either True or False. </br>Missing values should be replaced with False. |
| Takeaway Option | Nominal. </br>If take away is available. Either True or False. </br>Missing values should be replaced with False.|

In [5]:
#Importing relevant libraries and modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from category_encoders import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

#Setting display settings
pd.set_option('display.max_columns', None)

#Importing file with pd.read_csv
coffee = pd.read_csv("coffee.csv")

#Checking dtypes in columns
print(coffee.columns)
print(coffee.dtypes)

#Checking unique values in columns
unique_list=[]
for column in coffee.columns: 
    unique_list.append(coffee[column].unique())
print(unique_list)

#Checking for missing values in the dataframe
print(coffee.isna().any())

#Creating wrangler function
def wrangle(data):
    
    #Imputing missing values
    coffee["Rating"] = coffee["Rating"].fillna(0)
    coffee["Reviews"] = coffee["Reviews"].fillna(coffee["Reviews"].median())
    coffee["Dine in option"] = coffee["Dine in option"].fillna(False)
    coffee["Takeout option"] = coffee["Takeout option"].fillna(False)
    
    #Correcting incorrect data types
    coffee["Reviews"] = coffee["Reviews"].astype(str)
    coffee["Delivery option"] = coffee["Delivery option"].astype(str)
    coffee["Dine in option"] = coffee["Dine in option"].astype(str)
    coffee["Takeout option"] = coffee["Takeout option"].astype(str)
    
    #Encoding categorical columns
    label_encoder = OrdinalEncoder()
    label_encoder.fit_transform(coffee[["Reviews", "Price"]])

    return coffee

#Running wrangler
clean_data = wrangle(coffee)

#Checking clean_data
print(clean_data.head(10))
print(clean_data.dtypes)
print(clean_data.isna().any())

Index(['Region', 'Place name', 'Place type', 'Rating', 'Reviews', 'Price',
       'Delivery option', 'Dine in option', 'Takeout option'],
      dtype='object')
Region              object
Place name          object
Place type          object
Rating             float64
Reviews            float64
Price               object
Delivery option       bool
Dine in option      object
Takeout option      object
dtype: object
[array(['C', 'J', 'F', 'B', 'A', 'E', 'D', 'G', 'I', 'H'], dtype=object), array(['Dim Kavu', 'Коферум', 'Кофейня Світ Чаю', 'Кофейня Starcoff',
       'Кофейня "Friend Zone"', 'Racers Coffee Shop', 'Займемся Кофе',
       'Кофейня Rit Rit', "Кав'ярня My coffee",
       'LENЬ. Coffee & desserts.', 'SOVA COFFEE', 'Кава Тайм',
       'Skver кафе', 'Кафе на Георгіївській', 'Khosper',
       'Lekontina Шоколадна Майстерня', 'Lecker', 'Veterano Coffee',
       'VEIN', 'Coffee Drive', 'G COFFEE', 'Kavun', 'Buns Brew Bar',
       'Coffee House', '"Точка кофе"', 'Your Coffee', 'KOFEiN'

# Task 2 

The team at Java June believe that the number of reviews changes depending on the rating. 

Producing a table showing the difference in the median number of reviews by rating along with the minimum and maximum number of reviews to investigate this question for the team.

 - You should start with the data in the file 'coffee.csv'.

 - Your output should be a data frame named `reviews_by_rating`. 

 - It should include the three columns `rating`, `med_review`, `min_review`, `max_review`. 

 - Your answers should be rounded to 1 decimal place.   

In [6]:
#Building reviews_by_reting df with Reviews, and Rating columns
reviews_by_rating = coffee[["Rating", "Reviews"]]

#Using groupby and agreggation to obtain median, min, and max values of Reviews for each unique rating
reviews_by_rating = reviews_by_rating.groupby("Rating").agg({"Reviews": ["median", "min", "max"]}).reset_index()

#Setting column names according to requirements
reviews_by_rating.columns = ["rating", "med_review", "min_review", "max_review"]

#Setting rating as index
reviews_by_rating.set_index("rating", inplace=True)

#Viewing df to ensure everything is ok
print(reviews_by_rating)

        med_review min_review max_review
rating                                  
0.0          271.5      271.5      271.5
3.9            9.5       10.0        9.0
4.0          804.5     1439.0      170.0
4.1          452.5      189.0      716.0
4.2          497.0      385.0      609.0
4.3          221.5      102.0      646.0
4.4          536.0     1201.0      947.0
4.5          688.0     1195.0      998.0
4.6          693.0     1009.0      963.0
4.7          400.0      112.0       97.0
4.8          137.5       10.0       98.0
4.9           40.0       10.0       64.0
5.0           18.0       10.0      440.0


# Task 3

Fit a baseline model to predict the number of reviews a store will get.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `Place name` and `rating`. The rating column must be your predicted values.

In [7]:
#Importing training and validation data
train = pd.read_csv("train.csv")
X_test = pd.read_csv("validation.csv")

#Checking training and validation datasets
print(train.head(10))
print(X_test.head(10))

#Imputing missing values
def fill_na(ds):
    ds["Rating"] = ds["Rating"].fillna(0)
    if "Reviews" in ds.columns:
        ds["Reviews"] = ds["Reviews"].fillna(ds["Reviews"].median())
    ds["Dine in option"] = ds["Dine.in.option"].fillna(False)
    ds["Takeout option"] = ds["Takeout.option"].fillna(False) 
    
    return ds

train = fill_na(train)
X_test = fill_na(X_test)

#Separating features and target for training and testing
X_train = train.drop(["Reviews", "Place.name"], axis=1)
Y_train = train["Reviews"]
place_names = X_test["Place.name"]
X_test.drop("Place.name", axis =1, inplace=True)

#Columns to encode, split between ordinal encoding, non-ordinal encoding, and columns to passthrough
ohe_cols = ["Region", "Place.type"]
oe_cols = ["Price", "Delivery.option", "Dine.in.option", "Takeout.option"]
non_enc_columns = ["Rating"]

#Encoding nominal variables
encoder = ColumnTransformer(
    transformers=[('ohe', OneHotEncoder(), ohe_cols), ("oe", OrdinalEncoder(), oe_cols), ('num', 'passthrough', non_enc_columns)])

#Building function to run k-fold cross validation to get MSE of baseline model, using training subset
def bs_mse(feats, target):
    
    #Encoding categorical variables
    enc_feats = encoder.fit_transform(feats)
    
    #Initializaing LogReg
    tr_model = LinearRegression()
    
    #Defining cross-validation parameters
    cross_val = KFold(n_splits=5, shuffle=True)
    
    #Obtaining scores for different splits
    scores = cross_val_score(tr_model, enc_feats, target, cv=cross_val, scoring='neg_mean_squared_error')
    
    #Extracting the mean MSE from the folds
    return math.sqrt(-scores.mean()) 

#Building data pipeline for baseline model
pipeline = Pipeline([('encoder', encoder),('baseline', LinearRegression())
])

#Fitting the model to the training data
pipeline.fit(X_train, Y_train)

#Predicting reviews using baseline model
predictions = pipeline.predict(X_test)

base_result = pd.DataFrame({"Place Name": place_names, "rating": predictions})

print(base_result.head(20))
print(bs_mse(X_train, Y_train))

  Region                Place.name    Place.type  Rating  Reviews Price  \
0      C                  Dim Kavu        Others     4.6    206.0    $$   
1      C          Кофейня Світ Чаю   Coffee shop     5.0     11.0    $$   
2      C     Кофейня "Friend Zone"   Coffee shop     5.0     12.0    $$   
3      C        Racers Coffee Shop  Espresso bar     4.6    367.0    $$   
4      C             Займемся Кофе   Coffee shop     4.6    200.0    $$   
5      C           Кофейня Rit Rit   Coffee shop     4.6    292.0    $$   
6      C        Кав'ярня My coffee   Coffee shop     4.8     31.0    $$   
7      C  LENЬ. Coffee & desserts.   Coffee shop     4.8    125.0    $$   
8      C                Skver кафе          Cafe     4.9     18.0    $$   
9      C     Кафе на Георгіївській          Cafe     4.5    806.0     $   

   Delivery.option Dine.in.option Takeout.option  
0            False            NaN            NaN  
1            False            NaN           True  
2            False   

# Task 4

Fit a comparison model to predict the number of reviews a store will get.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `Place name` and `rating`. The rating column must be your predicted values.

In [8]:
#Importing training and validation data
train = pd.read_csv("train.csv")
X_test = pd.read_csv("validation.csv")

#Imputing missing values
def fill_na(ds):
    ds["Rating"] = ds["Rating"].fillna(0)
    if "Reviews" in ds.columns:
        ds["Reviews"] = ds["Reviews"].fillna(ds["Reviews"].median())
    ds["Dine in option"] = ds["Dine.in.option"].fillna(False)
    ds["Takeout option"] = ds["Takeout.option"].fillna(False) 
    
    return ds

train = fill_na(train)
X_test = fill_na(X_test)

#Separating features and target for training and testing
X_train = train.drop(["Reviews", "Place.name"], axis=1)
y_train = train["Reviews"]
place_names = X_test["Place.name"]
X_test.drop("Place.name", axis =1, inplace=True)

#Columns to encode, split between ordinal encoding, non-ordinal encoding, and columns to passthrough
ohe_cols = ["Region", "Place.type"]
oe_cols = ["Price", "Delivery.option", "Dine.in.option", "Takeout.option"]
non_enc_columns = ["Rating"]

#Encoding nominal variables
encoder = ColumnTransformer(
    transformers=[('ohe', OneHotEncoder(), ohe_cols), ("oe", OrdinalEncoder(), oe_cols), ('num', 'passthrough', non_enc_columns)])

#Building function to run k-fold cross validation to get MSE of Comparison model, using training subset
def cp_mse(feats, target):
    
    #Creating mse_list to store mse's for different parameters, pred_list to store parameters
    mse_list=[]
    param_list=[]
    
    #Encoding categorical variables
    enc_feats = encoder.fit_transform(feats)
    
    #Splitting train and test data within train dataset to assess performance
    ft_train, ft_test, tg_train, tg_test = train_test_split(feats, target, test_size=0.75)
    
    #Iterating hyperparameters to find optimal combination
    for max_depth in range(3, 10):
       for n_estimators in range(50, 500):
            for learning_rate in np.arange(0.01, 0.1, 0.01):
                
                #Initializing LogReg
                tr_model = GradientBoostingRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)
                
                #Fitting the model to the training data
                pipeline.fit(ft_train, tg_train)

                #Predicting reviews using baseline model
                y_pred = pipeline.predict(ft_test)
                
                #Appending parameters on param_list
                param_list.append((n_estimators, max_depth, learning_rate))
                
                #Calculating MSE for each set of parameters
                mse = mean_squared_error(tg_test, y_pred)
                
                #Appending mse values in mse_list
                mse_list.append(math.sqrt(mse))
                
                #Finding parameters that provide the lowest mse
                lowest_mse = mse_list.index(min(mse_list))
                best_parameters = param_list[lowest_mse]
                
    return best_parameters, min(mse_list)

#Defining the pipeline steps
pipeline = Pipeline([('encoder', encoder),('comparison', GradientBoostingRegressor(n_estimators=cp_mse(X_train, y_train)[0][0], max_depth=cp_mse(X_train, y_train)[0][1], learning_rate=cp_mse(X_train, y_train)[0][2]))])

#Fitting the model to the training data
pipeline.fit(X_train, y_train)

#Predicting reviews using baseline model
predictions = pipeline.predict(X_test)

compare_result = pd.DataFrame({"Place Name": place_names, "rating": predictions})
print(cp_mse(X_train, Y_train))
print(compare_result.head(20))

((50, 3, 0.01), 1134.0342313541917)
                            Place Name      rating
0                              Коферум  431.066828
1                     Кофейня Starcoff  723.060917
2                          SOVA COFFEE  500.625179
3                            Кава Тайм  431.066828
4                                 Dzhi  723.060917
5                            Koffishka  723.060917
6                               Verona  723.060917
7                        I love coffee  668.592364
8                             Дом Кофе  723.060917
9                       Prostir.coffee  668.592364
10                              HOTCUP  437.749661
11                          Art Coffee  437.749661
12         Мастерская "Кофе на разлив"  431.066828
13                            Dim Kavy  435.544157
14               Jays : Coffee Brewers  437.749661
15                   LittleTalk Coffee  723.060917
16                          Dobra Kava  723.060917
17           We Roast Coffee in Odessa  723.06